In [1]:
import numpy as np
import pandas as pd
import kagglehub
import torch
import nltk
from utils.text_datasets import get_basic_tweet_sentiment_dataset, get_poem_sentiment_dataset, get_advanced_tweet_sentiment_dataset
import pandas_profiling

/tmp/ipykernel_37442/772952456.py:7: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling


In [1]:
-99524 + -394 + 99918


0

# Utils

In [2]:
def get_text_statistics(dataset, remove_stopwords=False, top_n_words=10):
    num_samples = len(dataset)
    sample_lengths = [len(sample) for sample, _ in dataset]
    avg_length = np.mean(sample_lengths)

    words_in_set = [word for sample, _ in dataset for word in sample.split()]

    if remove_stopwords:
        stopwords = nltk.corpus.stopwords.words('english')
        words_in_set = [word for word in words_in_set if word not in stopwords]

    word_counts = pd.Series(words_in_set).value_counts()

    return {
        'num_samples': num_samples,
        'avg_length': avg_length,
        'num_words': len(words_in_set),
        'avg_words': len(words_in_set) / num_samples,
        'num_unique_words': len(word_counts),
        'unique_words': word_counts[:top_n_words],
    }

# Poem Sentiment dataset

In [3]:
whole_poem_dataset = torch.utils.data.ConcatDataset(get_poem_sentiment_dataset(None))

In [4]:
get_text_statistics(whole_poem_dataset, remove_stopwords=True)

{'num_samples': 1101,
 'avg_length': 38.328792007266124,
 'num_words': 4570,
 'avg_words': 4.150772025431426,
 'num_unique_words': 3190,
 'unique_words': thy       30
 like      23
 shall     19
 would     18
 thou      17
 yet       16
 see       16
 upon      13
 though    13
 three     12
 Name: count, dtype: int64}

# Basic Twitter Dataset

In [29]:
whole_basic_tweet_dataset = torch.utils.data.ConcatDataset(get_basic_tweet_sentiment_dataset(None))

/home/mikic202/miniconda3/envs/pt/lib/python3.11/site-packages/datasets/load.py:1491: FutureWarning: The repository for stanfordnlp/sentiment140 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/stanfordnlp/sentiment140
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [34]:
get_text_statistics(whole_basic_tweet_dataset, remove_stopwords=True)

{'num_samples': 1600498,
 'avg_length': 74.09207821565538,
 'num_words': 14047412,
 'avg_words': 8.776900689660343,
 'num_unique_words': 1350958,
 'unique_words': I        496739
 I'm       99579
 get       76748
 like      73315
 -         67121
 go        62987
 good      59797
 day       55756
 got       53890
 going     53248
 Name: count, dtype: int64}

# Andvanced tweet dataset

In [5]:
train_set, test_set, _, _ = get_advanced_tweet_sentiment_dataset(None)
whole_tweet_dataset = torch.utils.data.ConcatDataset([train_set, test_set])

In [15]:
[(data["text"], 1) for data, _ in whole_tweet_dataset]

text                 I`d have responded, if I were going
Country                                                0
Population -2020                               -0.008359
Land Area (Km²)                                -0.005149
Density (P/Km²)                                -0.147831
Age of User_0                                       True
Age of User_1                                      False
Age of User_2                                      False
Age of User_3                                      False
Age of User_4                                      False
Age of User_5                                      False
Time of Tweet_0                                     True
Time of Tweet_1                                    False
Time of Tweet_2                                    False
Name: 0, dtype: object
 I`d have responded, if I were going
<class 'str'>


/tmp/ipykernel_37442/1478275655.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(type(data[0]))


In [17]:
get_text_statistics([(data["text"], 1) for data, _ in whole_tweet_dataset], remove_stopwords=True)

{'num_samples': 31014,
 'avg_length': 68.28915973431353,
 'num_words': 264751,
 'avg_words': 8.536499645321468,
 'num_unique_words': 56783,
 'unique_words': I        9939
 I`m      1913
 get      1531
 like     1420
 -        1312
 day      1297
 go       1230
 good     1165
 got      1101
 going    1016
 Name: count, dtype: int64}

In [5]:
train_set.data["Age of User_0"]

0         True
1        False
2        False
3        False
4        False
         ...  
27475    False
27476    False
27477    False
27478    False
27479     True
Name: Age of User_0, Length: 27480, dtype: bool

In [9]:
profile = pandas_profiling.ProfileReport(pd.DataFrame(pd.concat([train_set.data, test_set.data])), title="Tweet Sentiment Analysis Dataset", explorative=True)
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]